First let's import the relevant module and initialize a neural db class.

In [ ]:
from thirdai import neural_db as ndb

db = ndb.NeuralDB(user_id="my_user")

At this point, the db is uninitialized. We can either initialize from scratch like this

In [ ]:
db.from_scratch()

Or load a checkpoint / base model as follows:

In [ ]:
db.from_checkpoint("path/to/checkpoint")

A database is useless if it doesn't contain anything. So let's insert things into it!

The insertion method takes in a list of `Document` objects. The `Document` interface is easily extendable. We plan to support some common formats out of the box, but for right now we'll show you how to extend the Document interface.

In [ ]:
from pathlib import Path
import pandas as pd
import shutil
    

class CSVDocument(ndb.Document):
    def __init__(self, path, strong_columns, weak_columns, reference_columns):
        self.path = Path(path)
        self._hash = ndb.utils.hash_file(path)
        self.df = pd.read_csv(path)
        self.strong_columns = strong_columns
        self.weak_columns = weak_columns
        self.reference_columns = reference_columns
    
    def hash(self) -> str:
        return self._hash
    
    def size(self) -> int:
        return len(self.df)
    
    def name(self) -> str:
        return self.path.name
    
    def strong_text(self, element_id: int) -> str:
        row = self.df.iloc[element_id]
        return " ".join([row[col] for col in self.strong_columns])
    
    def weak_text(self, element_id: int) -> str:
        row = self.df.iloc[element_id]
        return " ".join([row[col] for col in self.weak_columns])
    
    def reference(self, element_id: int) -> ndb.Reference:
        row = self.df.iloc[element_id]
        text = " ".join([row[col] for col in self.reference_columns])
        def show_fn(*args, **kwargs):
            print("Showing", text)
        return ndb.Reference(element_id, text, str(self.path.absolute()), {}, show_fn)
    
    def context(self, element_id: int, radius) -> str:
        row = self.df.iloc[element_id]
        return " ".join([row[col] for col in self.reference_columns])
    
    def save_meta(self, directory: Path):
        # Let's copy the original CSV file to the provided directory
        shutil.copy(self.path, directory)
    
    def load_meta(self, directory: Path):
        # Since we've moved the CSV file to the provided directory, let's make
        # sure that we point to this CSV file.
        self.path = directory / self.path.name


Phew! *Now* we insert the document.

In [ ]:
# @Tharun please change the parameters here.
csv_doc = CSVDocument(
    path="test_cs.csv", 
    weak_columns=["weak1", "wEaK2"], 
    strong_columns=["strong1", "stronk2"], 
    reference_columns=["ref1", "REF2"])

# Just like that!
db.add_documents([csv_doc])

# Note that this method does not throw since it catches all exceptions.
# To handle errors or progress, you can pass in handles like this
db.add_documents(
    documents=[csv_doc], 
    on_progress=lambda fraction: print(f"Progress at {fraction * 100}%"),
    on_error=lambda error_msg: print(f"Error! {error_msg}"),
    on_irrecoverable_error=lambda error_msg: print(f"Really bad error! {error_msg}"),
    on_success= lambda: print("SUCCESS!!!"))

# @Tharun I thought about changing this to be db.insert_into_db but I don't
# a real database would have a method called "insert_into_db" since people know
# it's a database. I strongly suggest that we keep it as is, or maybe rename it 
# to "insert", but I say let's avoid "db" in the method name. Plus, people
# are going to call the neural db object with the variable "db" since that's
# the example we give in this demo, so I think it will be clear enough that
# it's a DB!

Wait, we added the same document multiple times... is that going to be an issue?

Nope! Let's double-check by listing all of our sources.

In [ ]:
db.sources() # @Tharun on second thought we should probably call this documents() or list_documents() to be consistent.

Now let's search.

In [ ]:
search_results = db.search(
    query="STRONK", # @Tharun feel free to change the query.
    top_k=2,
    on_error=lambda error_msg: print(f"Error! {error_msg}"))

for result in search_results:
    print(result.text())
    print(result.source())
    print(result.metadata())
    result.show()

Now that we've covered the basics, let's go over some of NeuralDB's advanced features. The first one is text-to-result association. Suppose that out of the top k results above, you actually like the 3rd result best. You can tell the model exactly this by running the following snippet

In [ ]:
# @Tharun this is pending. DOn't run this cell.
db.text_to_result(text="STRONK", result_id=search_results[2].id())

Even more interesting than the first one is text-to-text association. This allows you to teach the model that two keywords, phrases, or concepts are related.

In [ ]:
# @Tharun please change as needed
# Maybe we can rename to associate_concepts or associate_texts or associate_phrases.
# But I prefer associate
# Or what about text_to_text?
db.associate(source="stronk", target="very stronk")

As usual, saving is a one-liner.

In [ ]:
db.save("path/to/checkpoint/")
# You can also see the progress
db.save("path/to/checkpoint/", on_progress=lambda fraction: print(f"{fraction}% done with saving."))

# Loading is just like we showed above, with an optional progress handler
db.from_checkpoint("path/to/checkpoint", on_progress=lambda fraction: print(f"{fraction}% done with loading."))

If you decide to start anew, we have just the right method for you.

In [ ]:
db.clear_documents()